In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle

In [2]:
data = pd.read_excel('movie data_new1.xlsx')
data.rename(columns={'Unnamed: 0': 'movie_id'}, inplace=True)
columns = ['Cast','Director','Genre','Title','Description']

In [3]:
def get_important_features(data):
    imp_features = []
    for i in range(0,data.shape[0]):
        imp_features.append(data['Title'][i]+' '+data['Director'][i]+' '+data['Genre'][i]+' '+data['Description'][i])
    return imp_features

In [4]:
data['imp_features'] = get_important_features(data)

In [11]:
data = data.drop(data[data["Title"]=='X'].index)

In [12]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['imp_features'])
print(tfidf_matrix.shape)

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

(980, 739)
[[1.         0.         0.0709564  ... 0.02377826 0.00640767 0.33983511]
 [0.         1.         0.00702245 ... 0.00710524 0.         0.        ]
 [0.0709564  0.00702245 1.         ... 0.13285159 0.00481644 0.02057527]
 ...
 [0.02377826 0.00710524 0.13285159 ... 1.         0.00487322 0.02081783]
 [0.00640767 0.         0.00481644 ... 0.00487322 1.         0.0056099 ]
 [0.33983511 0.         0.02057527 ... 0.02081783 0.0056099  1.        ]]


In [13]:
indices = pd.Series(data.index,index=data['Title']).drop_duplicates()
def get_recommendations(title,cosine_sim=cosine_sim):
    idx=indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores,key=lambda x:x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    movies = data['Title'].iloc[movie_indices]
    id = data['movie_id'].iloc[movie_indices]
    dict = {"Movies":movies,"id":id}
    final_df = pd.DataFrame(dict)
    final_df.reset_index(drop=True,inplace=True)
    return final_df
    

In [14]:
get_recommendations('The Godfather')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
data.info()
new = data.drop(columns=['Year of Release','Watch Time','Genre','Movie Rating','Metascore of movie','Director','Cast','Votes','Description'])

pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(cosine_sim,open('similarity.pkl','wb'))